<a href="https://colab.research.google.com/github/smccracken13/NBA-All-Star-Prediction-Project/blob/main/Capstone_Two_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [31]:
# Load Advanced.csv. This has the advanced individual player stats that I want.
nba_data = pd.read_csv('https://raw.githubusercontent.com/smccracken13/NBA-All-Star-Prediction-Project/main/Raw_Data/Advanced.csv')


In [32]:
# Load PlayerTotals.csv. This has more individual player stats that I want.
nba_player_totals = pd.read_csv('https://raw.githubusercontent.com/smccracken13/NBA-All-Star-Prediction-Project/main/Raw_Data/Player%20Totals.csv')

# drop duplicate columns from player totals, keep seas_id
print(nba_data.columns)
print(nba_player_totals.columns)
nba_player_totals = nba_player_totals.drop(['season','player_id','player', 'birth_year', 'pos','age','experience','lg', 'tm','g','mp','gs'],axis=1)
nba_player_totals.columns
# Merge two dataframes on seas_id
nba_data = pd.merge(nba_data, nba_player_totals, on ='seas_id',how='left')

Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'mp', 'per', 'ts_percent', 'x3p_ar',
       'f_tr', 'orb_percent', 'drb_percent', 'trb_percent', 'ast_percent',
       'stl_percent', 'blk_percent', 'tov_percent', 'usg_percent', 'ows',
       'dws', 'ws', 'ws_48', 'obpm', 'dbpm', 'bpm', 'vorp'],
      dtype='object')
Index(['seas_id', 'season', 'player_id', 'player', 'birth_year', 'pos', 'age',
       'experience', 'lg', 'tm', 'g', 'gs', 'mp', 'fg', 'fga', 'fg_percent',
       'x3p', 'x3pa', 'x3p_percent', 'x2p', 'x2pa', 'x2p_percent',
       'e_fg_percent', 'ft', 'fta', 'ft_percent', 'orb', 'drb', 'trb', 'ast',
       'stl', 'blk', 'tov', 'pf', 'pts'],
      dtype='object')


In [33]:
nba_data.columns
# Drop columns I don't want: ['birth_year','lg']
nba_data = nba_data.drop(['birth_year','lg'], axis=1)
# The NBA and ABA merged in 1976. The three-point line was added shortly after in 1979.
#I'll take only the rows after that so that I only have data from one league and fewer missing 3 point stats. That still leaves 40+ years of data.
nba_data=nba_data[nba_data['season']>1979]

In [34]:
# Load End of Season Teams.csv. This has the All-Star selections info that I want.
EOS_teams = pd.read_csv('https://raw.githubusercontent.com/smccracken13/NBA-All-Star-Prediction-Project/main/Raw_Data/End%20of%20Season%20Teams.csv')
# Drop seasons before NBA/ABA merge and 3-pt-line
EOS_teams = EOS_teams[EOS_teams['season']>1979]

# Each player/season has already been assigned a 'seas_id'.
# The only column I want from this df is 'type' and 'number_tm', which will tell me which players made the ALL-NBA team.
## Edit: I decided not to import 'number_tm' since my goal is to find All-Stars. I don't care which All-Star team they made.
# Drop all other columns
EOS_teams = EOS_teams[['seas_id','type']]

# Remove all rows that contain other awards like All-Rookie, All-Defense, etc... We only want All-NBA
EOS_teams = EOS_teams[EOS_teams['type']=='All-NBA']

In [35]:
# Merge two dataframes on seas_id, keeping all rows from advanced stats and adding all-star team data to those rows
nba_data = pd.merge(nba_data, EOS_teams, on ='seas_id',how='left')
# Add column 'all_star' and make it True if 'type'=='All-NBA', else false
nba_data['all_star']=np.where(nba_data['type']=='All-NBA',True,False)
# Drop 'type' column
nba_data = nba_data.drop(['type'],axis=1)
print(nba_data.columns)

Index(['seas_id', 'season', 'player_id', 'player', 'pos', 'age', 'experience',
       'tm', 'g', 'mp', 'per', 'ts_percent', 'x3p_ar', 'f_tr', 'orb_percent',
       'drb_percent', 'trb_percent', 'ast_percent', 'stl_percent',
       'blk_percent', 'tov_percent', 'usg_percent', 'ows', 'dws', 'ws',
       'ws_48', 'obpm', 'dbpm', 'bpm', 'vorp', 'fg', 'fga', 'fg_percent',
       'x3p', 'x3pa', 'x3p_percent', 'x2p', 'x2pa', 'x2p_percent',
       'e_fg_percent', 'ft', 'fta', 'ft_percent', 'orb', 'drb', 'trb', 'ast',
       'stl', 'blk', 'tov', 'pf', 'pts', 'all_star'],
      dtype='object')


In [36]:
# Load team summaries and get total home game attendance and win % for each team/season
team_wins = pd.read_csv('https://raw.githubusercontent.com/smccracken13/NBA-All-Star-Prediction-Project/main/Raw_Data/Team%20Summaries.csv')
team_wins=team_wins[['season','abbreviation','w','l','attend']]
# rename 'tm' column to match other df
team_wins = team_wins.rename(columns={'abbreviation':'tm'})
# create win % column
team_wins['tm_win_percent']=team_wins['w']/(team_wins['w']+team_wins['l'])
# drop 'w' and 'l' columns now that we have win %
team_wins=team_wins[['season','tm','attend','tm_win_percent']]
# drop seasons before NBA/ABA merge and 3-pt line
team_wins = team_wins[team_wins['season']>1979]
print(team_wins.head())
team_wins.shape

   season   tm    attend  tm_win_percent
0    2023  ATL  191799.0        0.545455
1    2023  BOS  210716.0        0.818182
2    2023  BRK  190804.0        0.521739
3    2023  CHI  198146.0        0.428571
4    2023  CHO  147860.0        0.285714


(1268, 4)

In [37]:
# Merge team_wins with nba_data. My goal is to add the correct season/tm data to each player in nba_data
# In other words, if a player in nba_data has a certain season/tm combo, I want to pull the matching 'attend_g' and 'tm_win_percent' and add it to that row
nba_data = pd.merge(nba_data,team_wins,how='left',on=['season','tm'])
# A quick check suggests the merge worked as I intended. The pacers had a 30.5% win percentage in the 2022 season
print(nba_data.loc[1000,'tm_win_percent'])
# Since the 2023 is still ongoing and all-stars have not been selected, I want to set this season aside. I may want to try to predict its all-stars later.
nba_data_2023=nba_data[nba_data['season']==2023]
nba_data =nba_data[nba_data['season']!=2023]

0.3048780487804878


In [38]:
# Column names, data types, counts, unique values, ranges
print(nba_data.info())
# Data types seem good. It's a little odd that age is a float instead of integer but I don't see that being a problem.
# I have a spreadsheet with clarifications of what each column means: https://docs.google.com/spreadsheets/d/1dZ4z-HfQ3cxOJyk0tX3fWSPrt5ubUhA7t4Z_6PJk5o4/edit?usp=sharing
# I plan to model for the True/False of the 'all_star' column. I could also consider modeling for the 'number_tm' column (there are 3 tiers of All-Stars)
# Count missing values and percentages
missing = pd.concat([nba_data.isnull().sum(), 100 * nba_data.isnull().mean()], axis=1)
missing.columns=['null count', '% null']
missing.sort_values(by=['null count'], ascending=False)
# I want to investigate why so many rows don't have 'attend' or 'tm_win'%'.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22467 entries, 476 to 22942
Data columns (total 55 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   seas_id         22467 non-null  int64  
 1   season          22467 non-null  int64  
 2   player_id       22467 non-null  int64  
 3   player          22467 non-null  object 
 4   pos             22467 non-null  object 
 5   age             22467 non-null  float64
 6   experience      22467 non-null  int64  
 7   tm              22467 non-null  object 
 8   g               22467 non-null  int64  
 9   mp              22467 non-null  float64
 10  per             22462 non-null  float64
 11  ts_percent      22364 non-null  float64
 12  x3p_ar          22350 non-null  float64
 13  f_tr            22350 non-null  float64
 14  orb_percent     22462 non-null  float64
 15  drb_percent     22462 non-null  float64
 16  trb_percent     22462 non-null  float64
 17  ast_percent     22462 non-nul

,null count,% null
x3p_percent,3765,16.757912
attend,2400,10.682334
tm_win_percent,2017,8.977612
ft_percent,1005,4.473227
x2p_percent,190,0.845685
fg_percent,117,0.520764
e_fg_percent,117,0.520764
x3p_ar,117,0.520764
f_tr,117,0.520764
ts_percent,103,0.458450


In [39]:
# I want to investigate why so many rows don't have 'attend' or 'tm_win'%'.
nba_data[nba_data['attend'].isna()][['season','tm','player','g','attend','tm_win_percent','all_star']]
# I've learned that 'TOT' means a player played for more than one team that year.
# Are there any missing team win % where team is not TOT?
nba_data[nba_data['tm_win_percent'].isna()]['tm'].value_counts()
# All missing team win % are from TOT. Set those win % to .500
nba_data['tm_win_percent'] = nba_data['tm_win_percent'].fillna(value=.500)

# fill attend with average attendance that season (Keep in mind this is 14% of that column. We may not want to use this column for our model)
seas_avg_attend = nba_data.groupby(by=['season'])['attend'].mean().reset_index()
seas_avg_attend.columns=['season','seas_avg_attend']
# print(season_avg_attend)
# These numbers look good except for 2020 and 2021. Those two seasons are much lower (especially 2021), due to COVID-19 changes to the season.
# I may need to replace those two seasons with an estimate of what they would have been in a normal year.

In [40]:
# Now that I have season_avg_attend, how do I get this into my NaN values of nba_data['attend']?
# add 'season_avg_attend' column to nba_data
nba_data = pd.merge(nba_data,seas_avg_attend,how='left',on=['season'])
# Fill missing 'attend' data with that season's average attendance
nba_data['attend'] = nba_data['attend'].fillna(value=nba_data['seas_avg_attend'])

In [41]:
# What else is missing?
missing1 = pd.concat([nba_data.isnull().sum(), 100 * nba_data.isnull().mean()], axis=1)
missing1.columns = ['null','null %']
missing1.sort_values(by=['null %'], ascending=False)

# what is going on wit those 5 rows that are missing 9 features: 'per'...'usg_percent'?
nba_data[nba_data['per'].isna()]
# Those 5 players all played 0 minutes those years.
nba_data[nba_data['mp']==0]
# In fact, they are the only rows in my data set to have 0 minutes played. I will drop them.
nba_data = nba_data[nba_data['mp']!=0]

In [42]:
# let's look into x3p %. I assume those players did not take any 3's, causing a divide by 0 situation.
nba_data[nba_data['x3p_percent'].isna()]['x3pa'].unique()
# Correct, none of those players attempted a 3. Let's make their 3% 0.
nba_data['x3p_percent'] = nba_data['x3p_percent'].fillna(value=0)

In [43]:
# likely the same issue with free_throw percentage
nba_data[nba_data['ft_percent'].isna()]['fta'].unique()
# Correct, none of those players attempted a free throw. Let's make their FT% 0.
nba_data['ft_percent'] = nba_data['ft_percent'].fillna(value=0)

In [44]:
# Let's check if it's the same issue for 2 pt %
nba_data[nba_data['x2p_percent'].isna()]['x2pa'].unique()
# Correct, none of those players attempted a 2 pointer. Let's make their x2p% 0.
nba_data['x2p_percent'] = nba_data['x2p_percent'].fillna(value=0)

In [45]:
# Do the same for fg% and efg%
nba_data[nba_data['fg_percent'].isna()]['fga'].unique()
# Correct, none of those players attempted a field goal. Let's make their fg% and efg% 0.
nba_data['fg_percent'] = nba_data['fg_percent'].fillna(value=0)
nba_data['e_fg_percent'] = nba_data['e_fg_percent'].fillna(value=0)
# field goal attempts are needed to calculate x3p_ar and f_tr as well
nba_data[['x3p_ar','f_tr']] = nba_data[['x3p_ar','f_tr']] .fillna(value=0)

In [46]:
# For true shooting %, players who have no field goal attampts and no free throws will lead to a 0 denominator
nba_data[nba_data['ts_percent'].isna()]['fga'].unique()
nba_data[nba_data['ts_percent'].isna()]['fta'].unique()
nba_data['ts_percent'] = nba_data['ts_percent'] .fillna(value=0)

In [47]:
# For turnover percent, players who have no FGA, FTA, and no turnovers will have a 0 denom
nba_data[nba_data['tov_percent'].isna()]['fga'].unique()
nba_data[nba_data['tov_percent'].isna()]['fta'].unique()
nba_data[nba_data['tov_percent'].isna()]['tov'].unique()
#since a 0 in this stat would be excellent, it seems more appropriate to set it to the mean
nba_data['tov_percent'] = nba_data['tov_percent'] .fillna(value=nba_data['tov_percent'].mean())

In [48]:
# let's look into turnover %
missing2 = pd.concat([nba_data.isnull().sum(), 100 * nba_data.isnull().mean()], axis=1)
missing2.columns = ['null','null %']
missing2.sort_values(by=['null %'], ascending=False)
# nba_data[nba_data['tov_percent'].isna()]
# It seems like these players did not attempt any shots those seasons, causing some of the advanced stats to divide by 0.

,null,null %
seas_id,0,0.0
season,0,0.0
fg,0,0.0
fga,0,0.0
fg_percent,0,0.0
x3p,0,0.0
x3pa,0,0.0
x3p_percent,0,0.0
x2p,0,0.0
x2pa,0,0.0


In [49]:
print(nba_data.info())
from google.colab import files
nba_data.to_csv('nba_data_cleaned.csv') 
files.download('nba_data_cleaned.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22462 entries, 0 to 22466
Data columns (total 56 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   seas_id          22462 non-null  int64  
 1   season           22462 non-null  int64  
 2   player_id        22462 non-null  int64  
 3   player           22462 non-null  object 
 4   pos              22462 non-null  object 
 5   age              22462 non-null  float64
 6   experience       22462 non-null  int64  
 7   tm               22462 non-null  object 
 8   g                22462 non-null  int64  
 9   mp               22462 non-null  float64
 10  per              22462 non-null  float64
 11  ts_percent       22462 non-null  float64
 12  x3p_ar           22462 non-null  float64
 13  f_tr             22462 non-null  float64
 14  orb_percent      22462 non-null  float64
 15  drb_percent      22462 non-null  float64
 16  trb_percent      22462 non-null  float64
 17  ast_percent 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>